In [2]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import scipy.stats as stats

Scenario: EcomX Retailers is a mid-sized online retail company specializing in electronics, clothing, and home goods. The company has been growing steadily over the past few years and wants to optimize its marketing and customer retention strategies. One key aspect of this strategy is to understand the Customer Lifetime Value (CLV) of its customers. By predicting the CLV, EcomX aims to identify high-value customers and tailor personalized marketing efforts to retain them, while also identifying low-value customers to manage resources effectively.

Objective: The objective of this assignment is to build and evaluate a machine learning model to predict the Customer Lifetime Value (CLV) of EcomX’s customers. You will need to identify and apply various data cleaning and preparation techniques, as well as select an appropriate model and evaluation criteria.

In [3]:
customer = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/customers_final.csv')
engagement = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/engagements_final.csv')
marketing = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/marketing_final.csv')
transactions = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/transactions_final.csv')

In [4]:
customer.head()

,customer_id,join_date,last_purchase_date,age,gender,location
0,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury
1,2,2021-09-08,2023-10-25,NaN,Male,Hillville
2,3,2021-06-01,2022-11-27,NaN,NaN,North Latoyatown
3,4,2022-01-01,2022-09-01,29.0,Male,Grossstad
4,5,2022-01-24,2023-06-02,NaN,Male,East Matthewfort


In [5]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   customer_id         10000 non-null  int64  
 1   join_date           10000 non-null  object 
 2   last_purchase_date  10000 non-null  object 
 3   age                 8991 non-null   float64
 4   gender              9467 non-null   object 
 5   location            10000 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 468.9+ KB


In [6]:
engagement.head()

,customer_id,number_of_site_visits,number_of_emails_opened,number_of_clicks
0,1,10,15,1
1,2,285,49,51
2,3,192,73,25
3,4,110,30,17
4,5,161,2,7


In [7]:
engagement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   customer_id              10000 non-null  int64
 1   number_of_site_visits    10000 non-null  int64
 2   number_of_emails_opened  10000 non-null  int64
 3   number_of_clicks         10000 non-null  int64
dtypes: int64(4)
memory usage: 312.6 KB


In [8]:
marketing.head()

,campaign_id,customer_id,response,promotion_type,campaign_date
0,1,1,No,Buy One Get One,2024-02-29
1,2,1,No,Discount,2024-01-24
2,3,1,No,Free Shipping,2024-03-05
3,4,1,Yes,Buy One Get One,2024-01-10
4,5,2,Yes,Free Shipping,2022-07-08


In [9]:
marketing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25200 entries, 0 to 25199
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   campaign_id     25200 non-null  int64 
 1   customer_id     25200 non-null  int64 
 2   response        25200 non-null  object
 3   promotion_type  25200 non-null  object
 4   campaign_date   25200 non-null  object
dtypes: int64(2), object(3)
memory usage: 984.5+ KB


In [10]:
transactions.head()

,transaction_id,customer_id,transaction_date,transaction_amount,product_category
0,1,1,2024-02-03,165.56,Clothing
1,2,1,2024-03-02,699.01,Home Goods
2,3,1,2024-03-12,146.86,Home Goods
3,4,1,2024-01-20,927.46,Electronics
4,5,1,2024-02-25,1395.87,Electronics


In [8]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129545 entries, 0 to 129544
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   transaction_id      129545 non-null  int64  
 1   customer_id         129545 non-null  int64  
 2   transaction_date    129545 non-null  object 
 3   transaction_amount  129545 non-null  float64
 4   product_category    129545 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 4.9+ MB


In [19]:
# step 1: make transactions and marketing indexes as customer_id for easy joining
transactions.set_index('customer_id', inplace=True)
marketing.set_index('customer_id', inplace=True)

In [20]:
# in other words > which marketing campaign was the user exposed to before they made a transaction?
# and did they respond positively or negatively to that?
# we need to convert key dates to datetime objects

marketing['campaign_date'] = pd.to_datetime(marketing['campaign_date'])
transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'])

In [44]:
# now we can use the merge() function in combination with the joins
# join the first 3 tables

transaction_level = customer.merge(marketing).join(transactions)

In [46]:
transaction_level

,customer_id,join_date,last_purchase_date,age,gender,location,campaign_id,response,promotion_type,campaign_date,transaction_id,transaction_date,transaction_amount,product_category
0,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,4,Yes,Buy One Get One,2024-01-10,NaN,NaT,NaN,NaN
1,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,2,No,Discount,2024-01-24,1.0,2024-02-03,165.56,Clothing
1,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,2,No,Discount,2024-01-24,2.0,2024-03-02,699.01,Home Goods
1,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,2,No,Discount,2024-01-24,3.0,2024-03-12,146.86,Home Goods
1,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,2,No,Discount,2024-01-24,4.0,2024-01-20,927.46,Electronics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25195,9998,2023-09-17,2024-01-30,39.0,Male,New John,25196,Yes,Discount,2023-09-20,NaN,NaT,NaN,NaN
25196,9999,2022-05-10,2022-07-15,31.0,Female,Andrewland,25198,No,Discount,2022-05-14,NaN,NaT,NaN,NaN
25197,9999,2022-05-10,2022-07-15,31.0,Female,Andrewland,25197,No,Buy One Get One,2022-06-02,NaN,NaT,NaN,NaN
25198,10000,2023-05-26,2023-09-01,67.0,Male,Hollytown,25199,No,Free Shipping,2023-06-12,NaN,NaT,NaN,NaN


In [47]:
transaction_level.reset_index(inplace=True)

In [48]:
# reset index
marketing.reset_index(inplace=True)

In [49]:
# merge function we'll use requires sorting
transaction_level.sort_values(by=['customer_id','transaction_date'], inplace=True)
marketing.sort_values(by=['customer_id','campaign_date'], inplace=True)

In [50]:
# make sure sorting is correct
transaction_level[transaction_level['customer_id']==1]

,index,customer_id,join_date,last_purchase_date,age,gender,location,campaign_id,response,promotion_type,campaign_date,transaction_id,transaction_date,transaction_amount,product_category
19,3,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,3,No,Free Shipping,2024-03-05,19.0,2021-09-28,347.76,Clothing
11,2,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,1,No,Buy One Get One,2024-02-29,11.0,2021-09-29,1866.56,Electronics
21,3,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,3,No,Free Shipping,2024-03-05,21.0,2021-11-04,153.24,Electronics
9,2,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,1,No,Buy One Get One,2024-02-29,9.0,2021-12-11,910.21,Electronics
13,2,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,1,No,Buy One Get One,2024-02-29,13.0,2022-04-30,800.64,Home Goods
7,2,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,1,No,Buy One Get One,2024-02-29,7.0,2022-05-05,443.99,Electronics
16,3,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,3,No,Free Shipping,2024-03-05,16.0,2022-07-01,68.35,Home Goods
20,3,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,3,No,Free Shipping,2024-03-05,20.0,2022-07-22,344.47,Electronics
12,2,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,1,No,Buy One Get One,2024-02-29,12.0,2022-09-06,684.94,Home Goods
10,2,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,1,No,Buy One Get One,2024-02-29,10.0,2022-10-20,42.29,Clothing


In [51]:
# make sure sorting is correct
marketing[marketing['customer_id']==1]

,index,customer_id,campaign_id,response,promotion_type,campaign_date
0,3,1,4,Yes,Buy One Get One,2024-01-10
1,1,1,2,No,Discount,2024-01-24
2,0,1,1,No,Buy One Get One,2024-02-29
3,2,1,3,No,Free Shipping,2024-03-05


In [52]:
# Custom function to find the most recent campaign before each transaction
def find_previous_campaign(transaction_row, campaigns):
    customer_campaigns = campaigns[campaigns['customer_id'] == transaction_row['customer_id']]
    customer_campaigns = customer_campaigns[customer_campaigns['campaign_date'] <= transaction_row['transaction_date']]
    if not customer_campaigns.empty:
        return customer_campaigns.iloc[-1]  # Return the most recent campaign
    else:
        return pd.Series([None] * campaigns.shape[1], index=campaigns.columns)

# Apply the custom function to each transaction
matched_campaigns = transaction_level.apply(find_previous_campaign, axis=1, campaigns=marketing)

# Merge the matched campaigns with transactions
df_merged = transaction_level.join(matched_campaigns.add_prefix('campaign_'))

In [53]:
# merge dataframe and check customer # 1
# we can see for each transaction, now we can see if there was a marketing campaign sent to the customer before they made the transaction, and whether they responded Yes or No
# if there was NOT a marketing campaign sent to them before they made the transaction, we get missing values > hint: don't impute these.
df_merged[df_merged['customer_id']==1]

,index,customer_id,join_date,last_purchase_date,age,gender,location,campaign_id,response,promotion_type,...,transaction_id,transaction_date,transaction_amount,product_category,campaign_index,campaign_customer_id,campaign_campaign_id,campaign_response,campaign_promotion_type,campaign_campaign_date
19,3,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,3,No,Free Shipping,...,19.0,2021-09-28,347.76,Clothing,NaN,NaN,NaN,None,None,NaT
11,2,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,1,No,Buy One Get One,...,11.0,2021-09-29,1866.56,Electronics,NaN,NaN,NaN,None,None,NaT
21,3,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,3,No,Free Shipping,...,21.0,2021-11-04,153.24,Electronics,NaN,NaN,NaN,None,None,NaT
9,2,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,1,No,Buy One Get One,...,9.0,2021-12-11,910.21,Electronics,NaN,NaN,NaN,None,None,NaT
13,2,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,1,No,Buy One Get One,...,13.0,2022-04-30,800.64,Home Goods,NaN,NaN,NaN,None,None,NaT
7,2,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,1,No,Buy One Get One,...,7.0,2022-05-05,443.99,Electronics,NaN,NaN,NaN,None,None,NaT
16,3,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,3,No,Free Shipping,...,16.0,2022-07-01,68.35,Home Goods,NaN,NaN,NaN,None,None,NaT
20,3,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,3,No,Free Shipping,...,20.0,2022-07-22,344.47,Electronics,NaN,NaN,NaN,None,None,NaT
12,2,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,1,No,Buy One Get One,...,12.0,2022-09-06,684.94,Home Goods,NaN,NaN,NaN,None,None,NaT
10,2,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,1,No,Buy One Get One,...,10.0,2022-10-20,42.29,Clothing,NaN,NaN,NaN,None,None,NaT


In [54]:
# double check marketing aligns
marketing[marketing['customer_id']==1]

,index,customer_id,campaign_id,response,promotion_type,campaign_date
0,3,1,4,Yes,Buy One Get One,2024-01-10
1,1,1,2,No,Discount,2024-01-24
2,0,1,1,No,Buy One Get One,2024-02-29
3,2,1,3,No,Free Shipping,2024-03-05


In [55]:
# double check transactions are correct
transaction_level[transaction_level['customer_id']==1]

,index,customer_id,join_date,last_purchase_date,age,gender,location,campaign_id,response,promotion_type,campaign_date,transaction_id,transaction_date,transaction_amount,product_category
19,3,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,3,No,Free Shipping,2024-03-05,19.0,2021-09-28,347.76,Clothing
11,2,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,1,No,Buy One Get One,2024-02-29,11.0,2021-09-29,1866.56,Electronics
21,3,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,3,No,Free Shipping,2024-03-05,21.0,2021-11-04,153.24,Electronics
9,2,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,1,No,Buy One Get One,2024-02-29,9.0,2021-12-11,910.21,Electronics
13,2,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,1,No,Buy One Get One,2024-02-29,13.0,2022-04-30,800.64,Home Goods
7,2,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,1,No,Buy One Get One,2024-02-29,7.0,2022-05-05,443.99,Electronics
16,3,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,3,No,Free Shipping,2024-03-05,16.0,2022-07-01,68.35,Home Goods
20,3,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,3,No,Free Shipping,2024-03-05,20.0,2022-07-22,344.47,Electronics
12,2,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,1,No,Buy One Get One,2024-02-29,12.0,2022-09-06,684.94,Home Goods
10,2,1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,1,No,Buy One Get One,2024-02-29,10.0,2022-10-20,42.29,Clothing


Data Cleaning